In [1]:
# !pip install accelerate
# !pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
# !pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 1.9 MB/s eta 0:00:00


In [3]:
import os
import nltk
from datasets import DatasetDict
from sklearn.feature_extraction import text
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
from transformers import TrainerCallback
from transformers import DataCollatorWithPadding

from peft import PeftModel, LoraConfig, get_peft_model

# Download necessary NLTK data
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
# if torch.cuda.is_available():
#     print("CUDA is available! ")
# else:
#     print("CUDA is not available.")

# print(torch.version.cuda)

CUDA is available! 
12.1


In [5]:
# from google.colab import drive
# drive.mount('/content/drive')


Mounted at /content/drive


In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "drive/MyDrive/data"

data = pd.read_json(os.path.join(data_dir, 'data.jsonl'), lines=True)
test_data = pd.read_json(os.path.join(data_dir, 'test_final.jsonl'), lines=True)
train_data = pd.read_json(os.path.join(data_dir, 'train_final.jsonl'), lines=True)
validation_data = pd.read_json(os.path.join(data_dir, 'validation_final.jsonl'), lines=True)


# Remove duplicates
test_data = test_data.drop_duplicates(subset=['text'])
train_data = train_data.drop_duplicates(subset=['text'])
validation_data = validation_data.drop_duplicates(subset=['text'])


## Loading BERT pre-trained model

In [29]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model

# Initialize the BERT tokenizer and model
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

print(model)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### Loading LoRA

In [30]:
from peft import LoraConfig, TaskType
from peft import get_peft_model

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,
    lora_alpha=64,
    lora_dropout=0.1
)

model = get_peft_model(model, lora_config)

## Apply LoRA to the model

In [31]:
model = get_peft_model(model, lora_config)
print(model)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): PeftModelForSequenceClassification(
      (base_model): LoraModel(
        (model): BertForSequenceClassification(
          (bert): BertModel(
            (embeddings): BertEmbeddings(
              (word_embeddings): Embedding(30522, 768, padding_idx=0)
              (position_embeddings): Embedding(512, 768)
              (token_type_embeddings): Embedding(2, 768)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (encoder): BertEncoder(
              (layer): ModuleList(
                (0-11): 12 x BertLayer(
                  (attention): BertAttention(
                    (self): BertSdpaSelfAttention(
                      (query): lora.Linear(
                        (base_layer): Linear(in_features=768, out_features=768, bias=True)
                        (lora_dropout): ModuleDict(
         

## Tokenize data

In [32]:
# Tokenize data
def tokenize_data(texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=128)

train_encodings = tokenize_data(train_data['text'].tolist())
val_encodings = tokenize_data(validation_data['text'].tolist())
test_encodings = tokenize_data(test_data['text'].tolist())


### Create dataset

In [33]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextDataset(train_encodings, train_data['label'].tolist())
val_dataset = TextDataset(val_encodings, validation_data['label'].tolist())
test_dataset = TextDataset(test_encodings, test_data['label'].tolist())

#### Train model

In [34]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [35]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=8,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    label_names=["labels"]
)

print("Training labels range: ", min(train_data['label']), "to", max(train_data['label']))
print("Validation labels range: ", min(validation_data['label']), "to", max(validation_data['label']))
print("Test labels range: ", min(test_data['label']), "to", max(test_data['label']))

print("Training data NaN values:", train_data.isnull().values.any())
print("Validation data NaN values:", validation_data.isnull().values.any())
print("Test data NaN values:", test_data.isnull().values.any())

 # Initialize Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print(trainer)

Training labels range:  0 to 5
Validation labels range:  0 to 5
Test labels range:  0 to 5
Training data NaN values: False
Validation data NaN values: False
Test data NaN values: False


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [36]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.667000,1.105909,0.635421,0.640370,0.659949,0.635421
2,1.102400,0.830107,0.717303,0.727980,0.770007,0.717303
3,0.833700,0.673142,0.786032,0.790681,0.813227,0.786032
4,0.622400,0.558230,0.828640,0.831873,0.847031,0.828640
5,0.563600,0.499338,0.847907,0.850791,0.866212,0.847907
6,0.508100,0.445958,0.866988,0.869067,0.879514,0.866988
7,0.460200,0.427995,0.870693,0.873193,0.885786,0.870693
8,0.439600,0.415371,0.877177,0.879257,0.889581,0.877177


TrainOutput(global_step=5328, training_loss=0.7202484743731158, metrics={'train_runtime': 1228.977, 'train_samples_per_second': 277.349, 'train_steps_per_second': 4.335, 'total_flos': 2.257716113761075e+16, 'train_loss': 0.7202484743731158, 'epoch': 8.0})

In [37]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.4153711199760437, 'eval_accuracy': 0.8771767321230085, 'eval_f1': 0.8792567765972886, 'eval_precision': 0.8895811299335925, 'eval_recall': 0.8771767321230085, 'eval_runtime': 8.8852, 'eval_samples_per_second': 607.53, 'eval_steps_per_second': 9.567, 'epoch': 8.0}


In [38]:
model_path = f'./my_trained_models/bert-lora_melhor'
trainer.save_model(model_path)

In [39]:
!pip install accelerate -U

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=8,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=test_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [40]:
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=-1)

In [44]:
accuracy = accuracy_score(test_dataset.labels, pred_labels)
precision, recall, f1, _ = precision_recall_fscore_support(test_dataset.labels, pred_labels, average='macro')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")


Accuracy: 0.8762045959970348
Precision: 0.8421108560268311
Recall: 0.9057695245621131
F1-Score: 0.8682699911835643
